# Nova ransomware – IR lead field notebook

> **Purpose**: This notebook is written in the voice of an Incident Response (IR) lead.  
> It helps me (1) clarify what “Nova ransomware” is, (2) list the log sources & artifacts I expect to see, and (3) run a repeatable IR plan playbook and basic log triage.

**How I use this**  
- I paste/attach exported logs (CSV/JSON) from my SIEM/XDR here.  
- I run the triage cells to quickly scope: *initial access → privilege escalation → lateral movement → ransomware execution → impact*.  
- I capture decisions, timestamps, and evidence references for the post-incident report.


## 1) What “Nova ransomware” refers to (working definition)

In public reporting, **Nova** is commonly described as a **ransomware-as-a-service (RaaS)** operation that emerged in **2025** and is linked to / rebranded from **RALord**. The payload has been reported as a **Rust executable** that encrypts files and drops a ransom note, and campaigns commonly include **double extortion** (encryption + data theft / leak-site pressure).

**IR-relevant characteristics I should expect to validate in my environment**
- **File extensions** added to encrypted files (several variants have been reported across sources):  
  - `.ralord`  
  - `.<orig_ext>.LORD`  
  - `.<orig_ext>.README-Nova`  
  - `.<orig_ext>.RNOVA`
- **Ransom note**: may instruct contact via chat/IM identifiers (e.g., qTox / Session / Telegram handles), and may not state a fixed ransom amount.
- **Operator workflow**: typical modern ransomware chain (initial access → discovery → credential access → lateral movement → defense evasion → data staging/exfil → encryption).

> **Note to self**: “Nova” is also used as a name by unrelated malware families in other contexts (e.g., stealers). I must confirm the artifacts match *this* ransomware cluster before I label the incident.


## 2) Evidence & log sources I want *immediately*

### Endpoint / EDR / Sysmon
- Process creation (parent/child), command lines, hashes
- File events: mass rename + extension changes, ransom note creation, high-rate file writes
- Registry changes (persistence, disable security tooling)
- Service creation / modification (remote exec tools, suspicious services)
- Driver loads (rare but important), tamper protection events

### Windows Event Logs
- Security: authentication and privilege events, log clearing, user/service creation
- System: service installs/stops, unexpected reboots, VSS activity (indirect)
- PowerShell logs (if enabled): Script Block, Module logging

### Identity / Directory / Cloud
- AD: new users/groups, group membership changes, delegation changes
- VPN / SSO: suspicious logins, impossible travel, anomalous device posture

### Network
- Firewall / proxy: unusual outbound to unknown infrastructure, TOR usage indicators, large uploads
- DNS: spikes in NXDOMAIN, newly registered domains, suspicious TXT queries (varies)
- East-west: SMB/RPC/RDP bursts from a single host, DC enumeration patterns

### File servers / NAS
- SMB audit logs, file access bursts, share permission changes
- Snapshot/backup deletions, backup job failures

### Email & Web (if initial access likely)
- Phishing links, attachments execution lineage, macro downloads


## 3) High-confidence host artifacts for ransomware execution (what I hunt first)

### “Looks like encryption” indicators
- A single host starts **touching thousands of files** in minutes (read/write/rename)
- Files renamed with one of the expected suffixes (see §1)
- A ransom note file is created across many directories
- CPU spikes + sustained disk I/O from an unknown binary, often from user profile/temp paths

### Common “pre-encryption” behavior I try to confirm
- Backup and recovery inhibition:
  - `vssadmin delete shadows /all /quiet`
  - `wmic shadowcopy delete`
  - `wbadmin delete catalog`
  - `bcdedit /set {default} recoveryenabled No` (varies)
- Defense evasion / kill tools:
  - `taskkill /F /IM <security_tool>.exe`
  - `sc stop <service>` / `net stop <service>`
- Lateral movement tools that often appear in ransomware intrusions:
  - PSExec-like remote service creation
  - Remote scheduled tasks
  - WMI remote process creation
  - RDP bursts / new RDP sessions

### Windows Event IDs I rely on (if enabled)
- **4688**: Process creation (Security)  
- **4624/4625**: Successful/failed logons  
- **4672**: Special privileges assigned  
- **4720/4722/4726**: User created/enabled/deleted  
- **4732/4728**: User added to local/domain group  
- **7045**: Service installed (System)  
- **1102**: Audit log cleared  
- **Sysmon 1/3/11/13/22**: process/network/file/registry/DNS (depending on config)

> **Note to self**: logging coverage varies. If 4688 is missing, lean on EDR telemetry and file server logs.


## 4) IR plan (ransomware-specific), written as my checklist

### Phase A — Identify & Stabilize (first hour)
1. Declare incident, start **incident log** (timestamps, decisions, who/what/when).  
2. **Isolate** suspected hosts (EDR network containment or VLAN quarantine).  
3. Preserve volatile evidence:
   - running processes, network connections, logged-on users
   - memory capture for key hosts (if feasible)
4. Confirm impact:
   - what is encrypted? endpoints vs file servers vs hypervisors?
   - is there a ransom note? what extension changes?
5. Decide on **containment boundary**:
   - block suspected C2/exfil destinations
   - restrict admin tools (PSExec/WMI), disable RDP if needed
6. Notify legal/comms/privacy; start regulatory clock if applicable.

### Phase B — Scope & Root Cause (same day)
1. Find **patient zero** and initial access path:
   - phishing? exposed VPN/RDP? stolen creds? supply chain?
2. Determine if data exfil likely occurred:
   - large outbound transfers, staging directories, cloud uploads
3. Identify the attacker’s footholds:
   - persistence, new users, scheduled tasks, services
4. Build a timeline: initial access → privilege escalation → lateral movement → encryption start.

### Phase C — Eradication
1. Remove persistence, reset credentials (tiered), rotate secrets/keys.  
2. Patch exploited surfaces and harden remote access.  
3. Reimage/restore compromised endpoints (prefer rebuild over “clean”).  
4. Validate that EDR policy prevents re-execution.

### Phase D — Recovery
1. Restore from known-good backups (offline/immutable preferred).  
2. Validate integrity before reconnecting to production network.  
3. Monitor for reinfection (heightened detection rules 2–4 weeks).

### Phase E — Lessons Learned
- What control failed? logging gaps? backup gaps? IAM gaps?
- Update playbooks, tabletop, and detection engineering backlog.


## 5) Data inputs for this notebook

Drop exports from your tools into a folder and point `DATA_DIR` at it. Recommended exports:
- `process_events.csv` (EDR/Sysmon/Security 4688-like)
- `file_events.csv` (EDR/Sysmon 11-like or file server audit)
- `auth_events.csv` (4624/4625 or IAM logs)
- `network_events.csv` (proxy/firewall/DNS summaries)

Expected minimal columns (map as needed):
- Process: `timestamp, host, user, process, parent_process, command_line, sha256`
- File: `timestamp, host, user, file_path, action, process`
- Auth: `timestamp, host, user, src_ip, logon_type, status`
- Network: `timestamp, host, user, dest_ip, dest_domain, dest_port, bytes_out`


In [ ]:

from pathlib import Path
import pandas as pd

DATA_DIR = Path("data")  # <-- change as needed
DATA_DIR.exists(), DATA_DIR.resolve()


In [ ]:

def load_optional_csv(name: str) -> pd.DataFrame:
    path = DATA_DIR / name
    if not path.exists():
        print(f"[!] Missing {path}. Create it or export from your tooling.")
        return pd.DataFrame()
    df = pd.read_csv(path)
    # normalize timestamp if present
    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
    return df

proc = load_optional_csv("process_events.csv")
files = load_optional_csv("file_events.csv")
auth = load_optional_csv("auth_events.csv")
net = load_optional_csv("network_events.csv")

(len(proc), len(files), len(auth), len(net))


## 6) Quick triage hunts (opinionated)

### 6.1 Hunt: Nova/RALord-like encrypted file suffixes
This is a fast way to confirm whether we’re dealing with the cluster that uses `.ralord`, `.LORD`, `.README-Nova`, `.RNOVA` suffixing patterns.


In [ ]:

SUSPECT_SUFFIXES = (".ralord", ".LORD", ".README-Nova", ".RNOVA")

if files.empty or "file_path" not in files.columns:
    print("[!] file_events.csv missing or missing file_path column")
else:
    hits = files[files["file_path"].astype(str).str.endswith(SUSPECT_SUFFIXES, na=False)].copy()
    display(hits.sort_values("timestamp").head(50))
    print("Total suffix hits:", len(hits))


### 6.2 Hunt: ransom note creation across directories
If you know the ransom note filename (or a unique phrase), add it below.


In [ ]:

RANSOM_NOTE_KEYWORD = "README"  # change if you know exact note name

if files.empty or "file_path" not in files.columns:
    print("[!] file_events.csv missing or missing file_path column")
else:
    note_hits = files[files["file_path"].astype(str).str.contains(RANSOM_NOTE_KEYWORD, case=False, na=False)].copy()
    # optionally narrow to create/write actions
    if "action" in note_hits.columns:
        note_hits = note_hits[note_hits["action"].astype(str).str.contains("create|write|rename", case=False, na=False)]
    display(note_hits.sort_values("timestamp").head(100))
    print("Possible ransom note hits:", len(note_hits))


### 6.3 Hunt: suspicious process patterns around encryption time
Goal: find processes with **high file event volume**, or processes spawning `conhost.exe` / console windows, or unusual binaries in user/temp paths.


In [ ]:

if files.empty or "process" not in files.columns:
    print("[!] file_events.csv missing or missing process column")
else:
    top_procs = (files.assign(process=files["process"].astype(str))
                      .groupby("process")
                      .size()
                      .sort_values(ascending=False)
                      .head(30))
    display(top_procs.to_frame("file_event_count"))


In [ ]:

# If you have process telemetry, look for conhost.exe and suspicious parent-child chains
if proc.empty or "process" not in proc.columns:
    print("[!] process_events.csv missing or missing process column")
else:
    conhost = proc[proc["process"].astype(str).str.lower().str.contains("conhost.exe", na=False)].copy()
    cols = [c for c in ["timestamp","host","user","process","parent_process","command_line","sha256"] if c in conhost.columns]
    display(conhost[cols].sort_values("timestamp").head(50))
    print("conhost-related events:", len(conhost))


### 6.4 Hunt: backup deletion / recovery inhibition commands
Search command lines for common patterns (adjust for your environment).


In [ ]:

SUSPICIOUS_CMD_PATTERNS = [
    "vssadmin delete shadows",
    "wmic shadowcopy delete",
    "wbadmin delete",
    "bcdedit",
    "wevtutil cl",          # log clearing
    "cipher /w",            # wipe free space (sometimes)
]

if proc.empty or "command_line" not in proc.columns:
    print("[!] process_events.csv missing or missing command_line column")
else:
    cl = proc["command_line"].astype(str).str.lower()
    mask = False
    for p in SUSPICIOUS_CMD_PATTERNS:
        mask = mask | cl.str.contains(p.lower(), na=False)
    hits = proc[mask].copy()
    cols = [c for c in ["timestamp","host","user","process","parent_process","command_line","sha256"] if c in hits.columns]
    display(hits[cols].sort_values("timestamp").head(200))
    print("Suspicious command hits:", len(hits))


### 6.5 Hunt: lateral movement indicators (high level)
This is intentionally generic: I’m looking for sudden spikes of SMB/RPC/RDP and many authentications from one source host.


In [ ]:

# Auth spike by source IP (or host) – requires src_ip column
if auth.empty or "src_ip" not in auth.columns or "timestamp" not in auth.columns:
    print("[!] auth_events.csv missing required columns (timestamp, src_ip)")
else:
    auth["day"] = auth["timestamp"].dt.floor("D")
    spikes = (auth.groupby(["day","src_ip"])
                  .size()
                  .sort_values(ascending=False)
                  .head(50))
    display(spikes.to_frame("auth_events"))


In [ ]:

# Network egress spike – requires bytes_out column
if net.empty or "bytes_out" not in net.columns or "timestamp" not in net.columns:
    print("[!] network_events.csv missing required columns (timestamp, bytes_out)")
else:
    net["day"] = pd.to_datetime(net["timestamp"], errors="coerce", utc=True).dt.floor("D")
    egress = (net.groupby(["day","host"])["bytes_out"]
                 .sum()
                 .sort_values(ascending=False)
                 .head(30))
    display(egress.to_frame("bytes_out_sum"))


## 7) SIEM query starters (copy/paste)

### Microsoft Sentinel / KQL (examples)
```kusto
// suspected Nova/RALord suffixes in file events
DeviceFileEvents
| where FolderPath has_any (".ralord", ".LORD", ".README-Nova", ".RNOVA")

// backup deletion commands
DeviceProcessEvents
| where ProcessCommandLine has_any ("vssadmin delete shadows", "wmic shadowcopy delete", "wbadmin delete", "wevtutil cl")
```

### Splunk SPL (examples)
```spl
index=endpoint (file_path="*.ralord" OR file_path="*.LORD" OR file_path="*.README-Nova" OR file_path="*.RNOVA")

index=endpoint process_command_line="*vssadmin*delete*shadows*" OR process_command_line="*wmic*shadowcopy*delete*"
```

> **Note**: adjust data models and field names to your environment.


## 8) Reporting placeholders (what I must capture)

- **First observed** encryption timestamp (UTC)  
- **Blast radius**: count of endpoints, servers, critical apps impacted  
- **Data exfil** assessment (evidence + confidence)  
- **Initial access hypothesis** and evidence  
- **Containment actions** taken (what/when/who)  
- **Eradication** actions (credential resets, reimages, patches)  
- **Recovery** plan, restore order, validation results  
- **External notifications**: legal/regulator/law enforcement/insurer  
